In [1]:
# Impordid
from src.transformers.models.bert.tokenization_bert import BertTokenizer
from src.transformers.models.bert.modeling_bert import BertEmbeddings, BertModel, BertForMaskedLM
import torch
from estnltk import Text
from estnltk.corpus_processing.parse_enc import parse_enc_file_iterator
from src.transformers.models.bert.configuration_bert import BertConfig
from transformers import AdamW
from tqdm.auto import tqdm
from transformers import pipeline
import numpy as np

In [2]:
# Tokeniseerija
tokenizer = BertTokenizer(vocab_file = "vocab_final.txt", vocab_file_form = "vocab_form.txt")

In [3]:
%%time
# Korpus
# Loeme tekstid sisse, laseme estnltk-l laused leida ning moodustame lausetest treening- ja testhulg
# https://github.com/estnltk/estnltk/blob/main/tutorials/corpus_processing/importing_text_objects_from_corpora.ipynb
input_file = "korpus/estonian_nc17.vert.01"
n = 2 # Mitu teksti korpusesse lugeda
korpus = []
l = 0
for text_obj in parse_enc_file_iterator(input_file):
    korpus.append(text_obj.text)
    if l > n:
        break
    l += 1

tekst = Text(" ".join(korpus)).tag_layer()
laused = []
for span in tekst.sentences:
    laused.append(tekst.text[span.start:span.end])
train =  laused[:int(0.8*len(laused))]
test = laused[int(0.8*len(laused)):]

CPU times: total: 500 ms
Wall time: 509 ms


In [4]:
# Dataseti loomine dataloaderi jaoks
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return self.encodings["input_ids"].shape[0]
    def __getitem__(self, i):
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [5]:
sample = tokenizer(train[0:2], max_length = 8, padding = "max_length", truncation = True, return_tensors = "pt")
tensor = sample.input_ids.detach().clone()
labels = tensor.detach().clone()
rand = torch.rand(tensor[:, :, 0].shape)
mask_arr = (rand < 0.15) * (tensor[:, :, 0] > 5)
for i in range(tensor[:, :, 0].shape[0]):
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    selection_masked = torch.where(mask_arr[i] == 0)[0].tolist()
    tensor[i, selection] = 4 # [MASK] tokeni ID mõlemas vocabis
    labels[i, selection_masked] = -100 # Et mudel arvutaks lossi ainult masked tokenite pealt

In [6]:
# Maskimine ja labelid
def mlm(tensor):
    labels = tensor.detach().clone()
    rand = torch.rand(tensor[:, :, 0].shape)
    mask_arr = (rand < 0.15) * (tensor[:, :, 0] > 5)
    for i in range(tensor[:, :, 0].shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        selection_masked = torch.where(mask_arr[i] == 0)[0].tolist()
        tensor[i, selection] = 4 # [MASK] tokeni ID mõlemas vocabis
        labels[i, selection_masked] = -100 # Et mudel arvutaks lossi ainult masked tokenite pealt
        
    return tensor, labels

In [7]:
# Andmestiku ettevalmistamine treenimiseks või mudelis kasutamiseks
def prepare_data(data):
    input_ids = []
    mask = []
    labels_vorm = []
    labels_lemma = []
    
    sample = tokenizer(data, max_length = 128, padding = "max_length", truncation = True, return_tensors = "pt")
    new_ids, new_labels = mlm(sample.input_ids.detach().clone())
    input_ids.append(new_ids)
    mask.append(sample.attention_mask)
    labels_vorm.append(new_labels[:, :, 1])
    labels_lemma.append(new_labels[:, :, 0])
    
    input_ids = torch.cat(input_ids)
    mask = torch.cat(mask)
    labels_vorm = torch.cat(labels_vorm)
    labels_lemma = torch.cat(labels_lemma)
    
    encodings = {
    "input_ids" : input_ids,
    "attention_mask" : mask,
    "labels_vorm" : labels_vorm,
    "labels_lemma" : labels_lemma,
    }
    
    dataset = Dataset(encodings)
    return dataset


# Dataloaderi loomine mudeli treenimiseks
dataloader_train = torch.utils.data.DataLoader(prepare_data(train), batch_size = 16, shuffle = True)

In [15]:
%%time
# Mudeli treenimine

config = BertConfig(
    vocab_size = tokenizer.vocab_size,
    vocab_size_form = tokenizer.vocab_size_form,
    tie_word_embeddings = False
)

model = BertForMaskedLM(config)
linear = torch.nn.Linear(tokenizer.vocab_size, tokenizer.vocab_size_form)
CE = torch.nn.CrossEntropyLoss()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)
optim = AdamW(model.parameters(), lr = 1e-4)
epochs = 2

for epoch in range(epochs):
    loop = tqdm(dataloader_train, leave = True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch["input_ids"]
        mask = batch["attention_mask"]
        labels_vorm = batch["labels_vorm"]
        labels_lemma = batch["labels_lemma"]
        output_lemma = model(input_ids, attention_mask = mask, labels = labels_lemma)
        loss_lemma = CE(output_lemma.logits.view(-1, tokenizer.vocab_size), labels_lemma.view(-1))
        output_vorm = linear(output_lemma.logits)
        loss_vorm = CE(output_vorm.view(-1, tokenizer.vocab_size_form), labels_vorm.view(-1))
        loss = loss_lemma + loss_vorm
        loss.backward()
        optim.step()
        loop.set_description(f"Epoch: {epoch}")
        loop.set_postfix(loss = loss.item())

  0%|          | 0/2 [00:00<?, ?it/s]

torch.Size([16, 128, 780])
torch.Size([12, 128, 780])


  0%|          | 0/2 [00:00<?, ?it/s]

torch.Size([16, 128, 780])
torch.Size([12, 128, 780])
CPU times: total: 7min 8s
Wall time: 55.9 s


In [19]:
output_lemma.logits.size()

torch.Size([12, 128, 50005])

In [10]:
# Testimine
dataloader_test = torch.utils.data.DataLoader(prepare_data(test), batch_size = 16)
kokku_labelid_vorm = np.array([-1])
kokku_labelid_lemma = np.array([-1])
kokku_ennustused_vorm = np.array([-1])
kokku_ennustused_lemma = np.array([-1])
#loop2 = tqdm(dataloader_test, leave = True) # Testandmed
loop2 = tqdm(dataloader_train, leave = True) # Treeningandmetel võiks accuracy tulla 1 kui epocheid piisavalt
for batch in loop2:
    input_ids = batch["input_ids"]
    labels_vorm = batch["labels_vorm"]
    labels_lemma = batch["labels_lemma"]
    
    output_lemma = model(input_ids).logits
    output_vorm = linear(output_lemma)
    indeksid = np.where(input_ids[:, :, 0].flatten() == 4)
    labels_vorm = labels_vorm.flatten()[indeksid].detach().numpy()
    labels_lemma = labels_lemma.flatten()[indeksid].detach().numpy()
    ennustused_vorm = np.argmax(output_vorm.flatten(end_dim = 1)[indeksid].detach().numpy(), axis = 1)
    ennustused_lemma = np.argmax(output_lemma.flatten(end_dim = 1)[indeksid].detach().numpy(), axis = 1)
    
    kokku_labelid_vorm = np.concatenate((kokku_labelid_vorm, labels_vorm))
    kokku_labelid_lemma = np.concatenate((kokku_labelid_lemma, labels_lemma))
    
    kokku_ennustused_vorm = np.concatenate((kokku_ennustused_vorm, ennustused_vorm))
    kokku_ennustused_lemma = np.concatenate((kokku_ennustused_lemma, ennustused_lemma))

  0%|          | 0/2 [00:00<?, ?it/s]

torch.Size([16, 128, 780])
torch.Size([12, 128, 780])


In [11]:
kokku_labelid_vorm
kokku_labelid_lemma

array([   -1,  1991,     6,  2051,   247, 11020,     6,   380, 37968,
           8,    89,   303,   523,    25,  2800,  1378,  8419,     6,
       10872,   251,   119,     8,  2019,  1009,   184,   588,     6,
         101,   263,  2894, 12219, 45401, 23473,    54,    25,    55,
         425,    25,    55, 23473,   139,   313,  3043,  1037,    55,
        2868,  8500,    78,    51,     6,   200, 19765,   487,     8,
         447,    51,   346,  6051,   207,    52,  2818], dtype=int64)

In [12]:
kokku_ennustused_vorm
kokku_ennustused_lemma

array([-1, 55, 55, 55, 55, 55,  8, 55, 55,  8, 55, 55, 55, 55, 55, 55, 55,
        8, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,  8,
       55, 55, 55, 55, 55, 55, 55,  8, 55, 55], dtype=int64)

In [13]:
sum(kokku_ennustused_vorm[1:] == kokku_labelid_vorm[1:])/len(kokku_labelid_vorm[1:])
sum(kokku_ennustused_lemma[1:] == kokku_labelid_lemma[1:])/len(kokku_labelid_lemma[1:])

0.06666666666666667

In [ ]:
#torch.save(model, "mudel.pth")